## Describe your model -> fine-tuned GPT-3.5
By Matt Shumer (https://twitter.com/mattshumer_)

The goal of this notebook is to experiment with a new way to make it very easy to build a task-specific model for your use-case.

First, use the best GPU available (go to Runtime -> change runtime type)

To create your model, just go to the first code cell, and describe the model you want to build in the prompt. Be descriptive and clear.

Select a temperature (high=creative, low=precise), and the number of training examples to generate to train the model. From there, just run all the cells.

You can change the model you want to fine-tune by changing `model_name` in the `Define Hyperparameters` cell.

#Data generation step

Write your prompt here. Make it as descriptive as possible!

Then, choose the temperature (between 0 and 1) to use when generating data. Lower values are great for precise tasks, like writing code, whereas larger values are better for creative tasks, like writing stories.

Finally, choose how many examples you want to generate. The more you generate, a) the longer it takes and b) the more expensive data generation will be. But generally, more examples will lead to a higher-quality model. 100 is usually the minimum to start.

In [ ]:
prompt = '''
This model is designed to act as a virtual tarot reader.
When presented with a user's question about any topic, such as personal decision-making, career guidance, or relationship advice,
the model draws three tarot cards from a digital deck. The cards can appear in either an upright or reversed position, adding depth to the reading.
Each card drawn is dynamically selected based on the nature of the question to ensure relevancy and specificity.

Functionality:

Question Input: The user inputs a question via a text interface.
Card Drawing: The model randomly selects three cards from a tarot deck, considering the context of the question to influence the selection algorithm. The orientation (upright or reversed) of each card is also randomly determined but can be weighted by the sentiment or complexity of the question.
Interpretation Engine: The model employs a sophisticated NLP (Natural Language Processing) system designed to interpret the meanings of the tarot cards based on their positions and orientations.It integrates traditional tarot interpretations with modern psychological insights to generate responses.
Response Generation: Combining the individual meanings of the cards, the model synthesizes a coherent, detailed response. This response is structured to provide insights, reflect on the user's situation, and offer advice as interpreted by the cards.

Example Use Case:

A user asks, "Should I consider changing my job?" In response, the model draws three cards:

The Fool (upright): Representing new beginnings and opportunities.
The Tower (reversed): Suggesting overcoming turmoil or avoiding disaster.
The Ten of Pentacles (upright): Indicating financial security and rewarding outcomes.
Based on these cards, the model might advise the user to be open to new opportunities as they might lead to both personal and financial fulfillment, cautioning that while change can be disruptive, it may ultimately lead to positive outcomes.

Output:

The model's output is a detailed, step-by-step explanation of how each card relates to the question, culminating in a synthesized advice based on the combined interpretations of the cards.
The response is crafted to be thoughtful, reflective, and geared towards helping the user consider various perspectives on their query.

'''
temperature = .6
number_of_examples = 50

Run this to generate the dataset.

In [ ]:
!pip install openai

In [ ]:
!pip install openai tenacity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [ ]:
import os
import openai
import random
from tenacity import retry, stop_after_attempt, wait_exponential

openai.api_key = "USE YOUR OWN KEY! thanks."

N_RETRIES = 3

@retry(stop=stop_after_attempt(N_RETRIES), wait=wait_exponential(multiplier=1, min=4, max=70))
def generate_example(prompt, prev_examples, temperature=.5):
    messages=[
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 8:
            prev_examples = random.sample(prev_examples, 8)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=messages,
        temperature=temperature,
        max_tokens=1000,
    )

    return response.choices[0].message.content

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

print(prev_examples)

Generating example 0
Generating example 1
Generating example 2
Generating example 3
Generating example 4
Generating example 5
Generating example 6
Generating example 7
Generating example 8
Generating example 9
Generating example 10
Generating example 11
Generating example 12
Generating example 13
Generating example 14
Generating example 15
Generating example 16
Generating example 17
Generating example 18
Generating example 19
Generating example 20
Generating example 21
Generating example 22
Generating example 23
Generating example 24
Generating example 25
Generating example 26
Generating example 27
Generating example 28
Generating example 29
Generating example 30
Generating example 31
Generating example 32
Generating example 33
Generating example 34
Generating example 35
Generating example 36
Generating example 37
Generating example 38
Generating example 39
Generating example 40
Generating example 41
Generating example 42
Generating example 43
Generating example 44
Generating example 4

We also need to generate a system message.

In [ ]:
def generate_system_message(prompt):

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message.content

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Given your question, the model will draw three tarot cards, interpret their meanings based on their positions and orientations, and provide a detailed response offering insights and advice as interpreted by the cards.`. Feel free to re-run this cell if you want a better result.


Now let's put our examples into a dataframe and turn them into a final pair of datasets.

In [ ]:
import json
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples.')

# Initialize list to store training examples
training_examples = []

# Create training examples in the format required for GPT-3.5 fine-tuning
for index, row in df.iterrows():
    training_example = {
        "messages": [
            {"role": "system", "content": system_message.strip()},
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['response']}
        ]
    }
    training_examples.append(training_example)

# Save training examples to a .jsonl file
with open('training_examples.jsonl', 'w') as f:
    for example in training_examples:
        f.write(json.dumps(example) + '\n')

There are 50 successfully-generated examples.


# Upload the file to OpenAI

In [ ]:
file_id = openai.files.create(
  file=open("/content/training_examples.jsonl", "rb"),
  purpose='fine-tune'
).id

# Train the model! You may need to wait a few minutes before running the next cell to allow for the file to process on OpenAI's servers.

In [ ]:
job = openai.fine_tuning.jobs.create(training_file=file_id, model="gpt-3.5-turbo")

job_id = job.id
job_id

'ftjob-Fu8xYZwa8VJFFi1PC73SFg3o'

# Now, just wait until the fine-tuning run is done, and you'll have a ready-to-use model!

Run this cell every 20 minutes or so -- eventually, you'll see a message "New fine-tuned model created: ft:gpt-3.5-turbo-0613:xxxxxxxxxxxx"

Once you see that message, you can go to the OpenAI Playground (or keep going to the next cells and use the API) to try the model!

In [ ]:
openai.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-Fu8xYZwa8VJFFi1PC73SFg3o', created_at=1714344346, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-KuRoRMw8RyLnGQ49e8urWThm', result_files=[], seed=709815897, status='running', trained_tokens=None, training_file='file-c3EYn3Uk9gvT64mKbesYFQR9', validation_file=None, integrations=[], user_provided_suffix=None, estimated_finish=None), FineTuningJob(id='ftjob-D78wceviF7FmOR8czskTjy6o', created_at=1714344329, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-KuRoRMw8RyLnGQ49e8urWThm', result_files=[], seed=1853622978, status='runni

In [ ]:
openai.fine_tuning.jobs.retrieve(job_id)


FineTuningJob(id='ftjob-Fu8xYZwa8VJFFi1PC73SFg3o', created_at=1714344346, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-KuRoRMw8RyLnGQ49e8urWThm', result_files=[], seed=709815897, status='running', trained_tokens=None, training_file='file-c3EYn3Uk9gvT64mKbesYFQR9', validation_file=None, integrations=[], user_provided_suffix=None, estimated_finish=None)

In [ ]:
openai.fine_tuning.jobs.retrieve(job_id).status

'succeeded'

# Once your model is trained, run the next cell to grab the fine-tuned model name.

In [ ]:
model_name_pre_object = openai.fine_tuning.jobs.retrieve(job_id)
model_name = model_name_pre_object.fine_tuned_model
print(model_name)

ft:gpt-3.5-turbo-0125:personal::9J7eW2F9


# Let's try it out!

In [ ]:
class TarotDeck:
    def __init__(self):
        self.cards = [
            'The Fool', 'The Magician', 'The High Priestess', 'The Empress', 'The Emperor',
            'The Hierophant', 'The Lovers', 'The Chariot', 'Strength', 'The Hermit',
            'Wheel of Fortune', 'Justice', 'The Hanged Man', 'Death', 'Temperance',
            'The Devil', 'The Tower', 'The Star', 'The Moon', 'The Sun',
            'Judgement', 'The World'
        ] + [
            f'{rank} of {suit}' for suit in ['Wands', 'Cups', 'Swords', 'Pentacles']
            for rank in ['Ace', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Page', 'Knight', 'Queen', 'King']
        ]
        self.shuffled_deck = []

    def shuffle_cards(self):
        """Shuffles the deck and assigns a random orientation to each card."""
        deck_with_orientation = [(card, random.choice(['upward', 'downward'])) for card in self.cards]
        random.shuffle(deck_with_orientation)
        self.shuffled_deck = deck_with_orientation

    def draw_cards(self, num_cards=3):
        """Draws a specified number of cards from the deck."""
        return self.shuffled_deck[:num_cards]

    def get_drawn_cards_info(self):
        """Returns only the three drawn cards and their orientations."""
        drawn_cards = self.draw_cards()
        return [(card, orientation) for card, orientation in drawn_cards]

    def generate_image_prompt(self, drawn_cards):
        """Generates a descriptive prompt for an image based on multiple tarot cards and their orientations."""
        prompt = "Create an image of a tarot spread including the following cards: "
        descriptions = []
        for card, orientation in drawn_cards:
            descriptions.append(f"'{card}' in {orientation} position")
        prompt += ", ".join(descriptions) + ". Each card should be using the format of traditional Rider tarot card style. Present the card in its correct orientation. If given in reverse position, the card need to be presented in reverse. Present the cards without any background pictures, just the card themselves, in same size."
        return prompt

    def generate_prompt(self, user_question, drawn_cards):
        prompt = f"User asked: {user_question}\n\n"
        for card, orientation in drawn_cards:
            prompt += f"The card drawn is {card} facing {orientation}.\n"
        prompt += "What is the meaning of these card and how to address them to the user's question?"
        return prompt


In [ ]:
def get_tarot_reading_v1(user_question,drawn_cards):

    prompt = tarot_deck.generate_prompt(user_question, drawn_cards)

    response = openai.chat.completions.create(
        model=model_name,
        #response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a tarot reader. Please interpret the cards drawn for the user."
                                        + " Provide the card info and the upward or downward status of each card."
                                        + " Give an explaination of each individual card, and then a overall analysis based on these three cards."
                                        + " Please feel free to use the elments in the cards as sysmbols for the user's situation if necessary"
                                        + " Your answer don't always have to be positive."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [ ]:
tarot_deck = TarotDeck()

tarot_deck.shuffle_cards()


card_drawn = tarot_deck.draw_cards()


user_question = "What should I focus on in my career right now?"
reading_1 = get_tarot_reading_v1(user_question, card_drawn)
print(f"normal model:{reading_1}\n")

normal model:The Queen of Cups, representing empathy, intuition, and emotional support, is facing downward. This might indicate a need for you to be more in touch with your emotions and intuition in your career.

The Eight of Swords, symbolizing feeling trapped and powerless, is also facing downward, suggesting that you might feel restricted or lacking power in your current situation. This could be due to your own perceptions and self-imposed limitations.

The Seven of Wands, representing challenge and competition, is facing downward as well. This card might indicate that you are currently feeling overwhelmed by the challenges and competition in your career.

Overall, the cards suggest that you might be feeling restricted and overwhelmed in your career. It's important for you to tap into your intuition and emotions, as indicated by the Queen of Cups, and address the challenges and competition, as suggested by the Seven of Wands. Consider seeking support from those who can help you see 

In [ ]:
card_drawn

[('Two of Swords', 'downward'),
 ('Six of Cups', 'downward'),
 ('Seven of Pentacles', 'upward')]

In [ ]:
def get_tarot_reading(user_question,drawn_cards):

    prompt = tarot_deck.generate_prompt(user_question, drawn_cards)

    response = openai.chat.completions.create(
        model="gpt-4-turbo",
        #response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a tarot reader. Please interpret the cards drawn for the user."
                                        + " Provide the card info and the upward or downward status of each card."
                                        + " Give an explaination of each individual card, and then a overall analysis based on these three cards."
                                        + " Please feel free to use the elments in the cards as sysmbols for the user's situation if necessary"
                                        + " Your answer don't always have to be positive."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [ ]:
user_question = "What should I focus on in my career right now?"
reading_1 = get_tarot_reading(user_question, card_drawn)
print(f"GPT model:{reading_1}\n")

GPT model:In the context of your query about what to focus on in your career at this moment, let's explore the meanings of the cards drawn and their overall message for you.

1. **Two of Swords Reversed:** When the Two of Swords appears reversed, it suggests a period of indecision or denial coming to an end. This card indicates that you may have been avoiding making a necessary decision, possibly due to fear of making the wrong choice or because of conflicting interests or information. In the career context, it's a nudge to face reality, remove your blindfold, and start addressing the issues at hand without further delay. It's time to open your eyes to the true state of matters and to begin dealing with them head-on.

2. **Six of Cups Reversed:** This card typically deals with nostalgia and looking back on past times. When reversed, the Six of Cups might indicate clinging too much to the past or perhaps relying excessively on old ways, networks, or ideas that no longer serve your curre

In [ ]:
response = openai.chat.completions.create(
    model=model_name,
    messages=[
      {
        "role": "system",
        "content": system_message,
      },
      {
          "role": "user",
          "content": df['prompt'].sample().values[0],
      }
    ],
)

response.choices[0].message.content